# Example reduction

In [ ]:
import sys
import os
sys.path.append(os.path.expanduser('~/git/LiquidsReflectometer/reduction'))

import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
import matplotlib.lines as mlines

#%matplotlib notebook
%matplotlib ipympl

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

In [ ]:
import mantid
import mantid.simpleapi as api

mantid.kernel.config.setLogLevel(3)

In [ ]:
print(mantid.__version__)
print(sys.version)

In [ ]:
import importlib

from lr_reduction import event_reduction, output, reduction_template_reader, template, workflow

In [ ]:
importlib.reload(workflow)
importlib.reload(output)
importlib.reload(event_reduction)

data_dir = os.path.expanduser('~/git/LiquidsReflectometer/reduction/data')
template_path = os.path.join(data_dir, 'template.xml')

for i in range(198409, 198417):
    ws = api.Load("REF_L_%s" % i)
    workflow.reduce(ws, template_path, output_dir=data_dir, pre_cut=1, post_cut=1, average_overlap=False)


In [ ]:
reduced_path = os.path.join(data_dir, 'reference_rq.txt')
if os.path.isfile(reduced_path):
    _data = np.loadtxt(reduced_path).T

reduced_path = os.path.join(data_dir, 'REFL_198409_combined_data_auto.txt')
if os.path.isfile(reduced_path):
    _refl = np.loadtxt(reduced_path).T

fig, ax = plt.subplots(figsize=(10,5))
plt.errorbar(_refl[0], _refl[1]*_refl[0]**4, yerr=_refl[2]*_refl[0]**4, markersize=4, marker='.', linestyle='', label='new reduction')
plt.errorbar(_data[0], _data[1]*_data[0]**4, yerr=_data[2]*_data[0]**4, markersize=4, marker='', linestyle='-', label='reference')

plt.legend()
plt.xlabel(r'q [$1/\AA$]')
plt.ylabel('R(q)')
ax.set_yscale('log')
ax.set_xscale('log')
plt.show()

if len(_data[1])==len(_refl[1]):
    # dQ is computed for each run in the new implementation
    fig, ax = plt.subplots(figsize=(10,5))
    plt.plot(_refl[0], _refl[3]/_refl[0], label="new_reduction")
    plt.plot(_data[0], _data[3]/_data[0], label="reference")

    plt.xlabel(r'q [$1/\AA$]')
    plt.ylabel(r'$\Delta q$')
    ax.set_yscale('linear')
    ax.set_xscale('log')
    plt.show()
else:
    print("len(ref) = %s; len(new) = %s" % (len(_data[1]), len(_refl[1])))

for i in range(4):
    print(np.sum(_data[i]-_refl[i]))


In [ ]:
importlib.reload(workflow)
importlib.reload(output)
importlib.reload(event_reduction)
importlib.reload(reduction_template_reader)

data_dir = os.path.expanduser('~/git/LiquidsReflectometer/reduction/data')
template_path = os.path.join(data_dir, 'template.xml')



def write_template(template_file, output_dir):
    #self.data_peak_range = [140, 150]
    #self.background_roi = [137, 153,100, 200]

    with open(template_file, "r") as fd:
        xml_str = fd.read()
        data_sets = reduction_template_reader.from_xml(xml_str)

        # Left background
        for i in range(8):
            print("Peak: %s" % data_sets[i].data_peak_range)
            data_sets[i].background_roi[0] = data_sets[i].data_peak_range[0]-5
            data_sets[i].background_roi[1] = data_sets[i].data_peak_range[0]

        xml_str = reduction_template_reader.to_xml(data_sets)
        with open(os.path.join(output_dir, 'template_left.xml'), 'w') as fd:
            fd.write(xml_str)

        # Right background
        for i in range(8):
            data_sets[i].background_roi[0] = data_sets[i].data_peak_range[1]
            data_sets[i].background_roi[1] = data_sets[i].data_peak_range[1]+5

        xml_str = reduction_template_reader.to_xml(data_sets)
        with open(os.path.join(output_dir, 'template_right.xml'), 'w') as fd:
            fd.write(xml_str)

        # Invalid background
        for i in range(8):
            data_sets[i].background_roi[0] = data_sets[i].data_peak_range[0]+1
            data_sets[i].background_roi[1] = data_sets[i].data_peak_range[1]-1

        xml_str = reduction_template_reader.to_xml(data_sets)
        with open(os.path.join(output_dir, 'template_invalid.xml'), 'w') as fd:
            fd.write(xml_str)


write_template(template_path, data_dir)
template_path_left = os.path.join(data_dir, 'template_left.xml')
template_path_right = os.path.join(data_dir, 'template_right.xml')
template_path_invalid = os.path.join(data_dir, 'template_invalid.xml')

for i in range(198409, 198417):
    ws = api.Load("REF_L_%s" % i)
    workflow.reduce(ws, template_path_invalid, output_dir=data_dir, pre_cut=1, post_cut=1, average_overlap=False)



In [ ]:
importlib.reload(workflow)
importlib.reload(output)
importlib.reload(template)
importlib.reload(event_reduction)
importlib.reload(reduction_template_reader)

# IPTS-26010
# 192872, 191674, 190189, 187797: quartz
# 192879, 190202, 187804: inverted quartz
# 193204: SiO2

# IPTS-29196
# 201364: quartz
# 201074: D2O
# 201083: Mo in cell

# IPTS-30101
# 197900: quartz

# IPTS-29815
# 201446: Rana

# IPTS-28210
# 193773: quartz

# Frank IPTS-28412
# 191499 (cell up to 0.4)

run_number = 201282
IPTS = 29196

run_number = 194385
IPTS = 28342

data_dir = os.path.expanduser('~/git/LiquidsReflectometer/reduction/data')
template_path = "/SNS/REF_L/IPTS-%s/shared/autoreduce/REF_L_%s_auto_template.xml" % (IPTS, run_number)
template_path = "/SNS/REF_L/IPTS-28342/shared/autoreduce/194391-no-bck.xml"


for i in range(run_number, run_number+7):
    ws = api.Load("REF_L_%s" % i)
    thi = ws.getRun().getProperty("thi").value[0]
    workflow.reduce(ws, template_path, output_dir=data_dir, pre_cut=1, post_cut=1, average_overlap=False, q_summing=False)


In [ ]:
reduced_path = "/SNS/REF_L/IPTS-%s/shared/autoreduce/REFL_%s_combined_data_auto.txt" % (IPTS, run_number)

if os.path.isfile(reduced_path):
    _data = np.loadtxt(reduced_path).T

reduced_path = os.path.join(data_dir, 'REFL_%s_combined_data_auto.txt' % run_number)
if os.path.isfile(reduced_path):
    _refl = np.loadtxt(reduced_path).T

fig, ax = plt.subplots(figsize=(10,5))
plt.errorbar(_refl[0], _refl[1], yerr=_refl[2], markersize=4, marker='.', linestyle='', label='new reduction')
plt.errorbar(_data[0], _data[1], yerr=_data[2], markersize=4, marker='', linestyle='-', label='old')

plt.legend()
plt.xlabel(r'q [$1/\AA$]')
plt.ylabel('R(q)')
ax.set_yscale('log')
ax.set_xscale('log')
plt.show()
